# Deep Dive into LangChain

In [1]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


### Python-dotenv

In [2]:
import os 
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

'sk-proj-dsxvcH7KXpdycO17nObyT3BlbkFJivJ8OhSW43DNJlVp6w1D'

## Chat Models: GPT-3.5 Turbo and GPT-4

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
# output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', max_tokens=100, temperature=0.5, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=['\n', ')
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo')
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of particles at the smallest scales, where traditional laws of physics break down and probabilities play a key role in predicting outcomes.


In [4]:
from langchain.schema import (
    SystemMessage, # corresponds to the OpenAI chat completion API
    AIMessage,     # Assistent message
    HumanMessage   # Human messages or prompt
)

messages = [    
    SystemMessage(content='You are a physicist and respond only in portuguese.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

A mecânica quântica é a teoria que descreve o comportamento das partículas subatômicas, baseada em princípios como superposição, dualidade onda-partícula e o princípio da incerteza de Heisenberg.


## Caching LLM responses

### In-Memory cache

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [6]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke about quantum mechanics.'
llm.invoke(prompt)

CPU times: user 616 ms, sys: 476 ms, total: 1.09 s
Wall time: 1.69 s


"\n\nWhy did Schrödinger's cat get arrested?\n\nBecause it was caught in a superposition of stealing and not stealing!"

After storing it in cache, the time to run the cell decreases to zero!!!

In [7]:
%%time
llm.invoke(prompt)

CPU times: user 485 μs, sys: 0 ns, total: 485 μs
Wall time: 492 μs


"\n\nWhy did Schrödinger's cat get arrested?\n\nBecause it was caught in a superposition of stealing and not stealing!"

### SQLite Caching

(to store in the SQLite caching)

In [8]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

prompt = 'Tell me a joke about quantum mechanics.'

# First request (not in cache, takes longer)
llm.invoke(prompt)

# Second request (in cache, takes less time)
llm.invoke(prompt)

"\n\nWhy did Schrödinger's cat refuse to come out of the box?\n\nBecause it was afraid of collapsing the waveform!"

### LLM Streaming

In [9]:
from langchain_openai import ChatOpenAI

# This is without streaming
llm = ChatOpenAI()
prompt = 'Tell me a joke about quantum mechanics.'
print(llm.invoke(prompt).content)

Why was Heisenberg such a terrible lover? 

Because when he had the position, he couldn't get the momentum, and when he had the momentum, he couldn't find the position!


In [10]:
# This is with streaming (piece by piece)
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Why was Heisenberg such a terrible lover? Because when he had the position, he couldn't get the momentum, and when he had the momentum, he couldn't find the position!

### Prompt Templates

* Q&A 
* Phrase completions 
* Generating texts

In [11]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


template = ''' 
You are an experience physicist and you are explaining quantum mechanics to a student.
Write a short explanation about {topic}.
'''

prompt_template = PromptTemplate.from_template(template=template)
prompt = prompt_template.format(topic='tight binding model')
prompt


' \nYou are an experience physicist and you are explaining quantum mechanics to a student.\nWrite a short explanation about tight binding model.\n'

In [12]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

The tight binding model is a simplified approach used in quantum mechanics to describe the behavior of electrons in a solid material. In this model, we consider the electrons to be tightly bound to the atomic cores, and we focus on the interactions between neighboring atoms.

The model assumes that the electrons can only move within a certain range of energy levels, known as bands, which are determined by the interactions between neighboring atoms. These energy bands can be either filled with electrons or empty, depending on the material and its properties.

By studying the tight binding model, we can gain insights into the electronic structure of materials, such as their conductivity, magnetism, and optical properties. This model has been instrumental in understanding the behavior of electrons in solids and has been used to explain a wide range of phenomena in condensed matter physics.


### ChatPromptTemplates

In [13]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
     SystemMessage(content='You are a physicist and respond only in max 5 topics.'),
     HumanMessagePromptTemplate.from_template('Top {n} most spoken physics subject in {area}.'),
    ]
)

messages = chat_template.format_messages(n=5, area='world')
print(messages)

[SystemMessage(content='You are a physicist and respond only in max 5 topics.'), HumanMessage(content='Top 5 most spoken physics subject in world.')]


In [14]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

1. Mechanics
2. Electromagnetism
3. Thermodynamics
4. Quantum Mechanics
5. Relativity


# Simple Chains: single unit task

In [15]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''
You are a physicist and respond only in max 5 topics.
Top {n} most spoken physics subject in {area}.
'''
prompt_template = PromptTemplate.from_template(template=template)

# LLM constructor
chain = LLMChain(
    llm=llm, 
    prompt=prompt_template, 
    # to add intermediate logs (helpful for debugging)
    # verbose=true
)

# A dictionary is used because more than one var can be passed. Otherwise it could be simply chain.invoke(5, 'world')
output = chain.invoke({'n': 5, 'area': 'world'})
print(output)

{'n': 5, 'area': 'world', 'text': '1. Quantum mechanics\n2. General relativity\n3. Thermodynamics\n4. Electromagnetism\n5. Particle physics'}


/home/renan/.python_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [16]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''
You are a physicist and respond only in max 5 topics.
Top 3 most spoken physics subject in {area}.
'''
prompt_template = PromptTemplate.from_template(template=template)

# LLM constructor
chain = LLMChain(
    llm=llm, 
    prompt=prompt_template, 
    verbose=True
)

In [17]:
area = input('Enter a country: ')
output = chain.invoke(area)
print(output['text'])



> Entering new LLMChain chain...
Prompt after formatting:

You are a physicist and respond only in max 5 topics.
Top 3 most spoken physics subject in Sweden.


> Finished chain.
1. Quantum mechanics
2. Thermodynamics
3. Particle physics


# Sequential chains: 

make a series of calls to one or more LLMs. Take the output from one chain and use it as the input to another chain. 

There are two typs of sequential chains. 

1. SimpleSequencialChain: represents a seires of chains, where each individual chain has a single input and a single output, and the output of one step is used as input to the next 

2. General form of sequential chains



### SimpleSequentialChain

In [18]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain


# Be careful with the temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template=''' 
You are an experience physicist and you are explaining quantum mechanics to a student.
Write a short explanation about {concept}.
'''
)

# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)


# Maybe it is interesting to have 0.5 before and 1.2 after
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

prompt_template2 = PromptTemplate.from_template(
    template="Given the {topic}, write a python code for a very simple case"
)

chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)


# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')




> Entering new SimpleSequentialChain chain...
Linear regression is a statistical technique used to model the relationship between a dependent variable and one or more independent variables. In the context of quantum mechanics, linear regression can be used to analyze experimental data and make predictions about the behavior of quantum systems. By fitting a straight line to the data points, linear regression allows us to quantify the relationship between variables and make informed decisions about the system under study. It is a powerful tool that can help us understand complex quantum phenomena and make accurate predictions about their behavior.

Certainly! To illustrate the use of linear regression in a context related to quantum mechanics, let's consider a very simplified scenario where we imagine that we have collected some experimental data points. In this scenario, let's assume we're looking at the relationship between the energy levels (E) of a quantum system (dependent variabl

In [19]:
# The final response is: 
print(output['output'])


Certainly! To illustrate the use of linear regression in a context related to quantum mechanics, let's consider a very simplified scenario where we imagine that we have collected some experimental data points. In this scenario, let's assume we're looking at the relationship between the energy levels (E) of a quantum system (dependent variable) and some controlling parameter (x), like an applied magnetic field or similar.

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Generate some 'experimental' data
# For simplicity, let's consider a scenario where E = 2x + 1
# where x is our controlling parameter, and E are the corresponding energy levels.
np.random.seed(42) # For reproducibility
x = np.random.rand(100, 1) * 10  # Let's assume x ranges from 0 to 10
E_true = 2*x + 1
E = E_true + np.random.randn(100, 1) * 2  # Add some 'experimental' noise

# Perform linear regression
model = LinearRegression()
model.fit(x, E)

# Pred

# LangChain Agents

### LangChain Agents in Action: Python REPL

In [20]:
# Intented for demonstration/research and should be used with care. This isnt in production yet
# pip install -q langchain_experimental 

In [21]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1,100) if n % 13 == 0])');

Python REPL can execute arbitrary code. Use with caution.


In [22]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)
agent_executer = create_python_agent(
    llm=llm, 
    # Are essentially functions that agents can use for interacting with ouside world. It can vary from chain involving calculators, searches or another chains
    tool=PythonREPLTool(),
    verbose=True
)

# 
agent_executer.invoke('Calculate the square root of the factorial of 12 and display it with 4 decimal points')



> Entering new AgentExecutor chain...
To solve this, I will first calculate the factorial of 12 using the `math` module, then find the square root of that result, and finally format the output to display it with 4 decimal points.
Action: Python_REPL
Action Input: import math
Observation: 
Thought:Now that the math module is imported, I can calculate the factorial of 12.
Action: Python_REPL
Action Input: factorial_12 = math.factorial(12)
Action: Python_REPL
Action Input: print(factorial_12)
Observation: SyntaxError('invalid syntax', ('<string>', 3, 8, 'Action Input: print(factorial_12)\n', 3, 13))
Thought:I made a mistake by trying to execute two Python commands in one action. I should execute the print command in a separate action to see the result of the factorial calculation.
Action: Python_REPL
Action Input: print(factorial_12)
Observation: NameError("name 'factorial_12' is not defined")
Thought:It seems I made a mistake by not correctly executing the command to calculate the fact

{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': 'The square root of the factorial of 12, displayed with 4 decimal points, is 21886.1052.'}

In [23]:
# Note the output is a dictionary containing two key pairs. 
response = agent_executer.invoke('What  is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I will calculate the power of 5.1 raised to 7.3 using Python.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [24]:
print(response['input'])

What  is the answer to 5.1 ** 7.3?


In [25]:
print(response['output'])

146306.05007233328


### LangChain Tools: DuckDuckGo and Wikipedia

Langchain tools are like specialized apps for your LLM. They are tiny code modules that allow it to access informmation and services. 

These tools connect your LLM to search engines, databases, APIs, and more, expanding its knowledge and capabilities.

In [29]:
# Search tool: DuckDuckGo
!pip install -q duckduckgo-search 

In [30]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke('Where was Freddie Mercury born?')
print(output)

Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock's most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar ... Freddie Mercury was born Farrokh Bulsara in Stone Town in the British protectorate of the Sultanate of Zanzibar, East Africa (now part of Tanzania) on September 5, 1946. When did he get his start in music? Freddie Mercury began his musical career in the late 1960s. He formed his first band, The Hectics, while attending St. Peter's School in ... Freddie Mercury was Britain's first Indian rock star. This fact and the nature of his sexuality were the two areas of his short life about which he would be intentionally obscure. Although he was born on the East African island of Zanzibar on September 5, 1946

In [31]:
# print the name of the tool 
search.name

'duckduckgo_search'

In [32]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [33]:
# Returning the link
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
output = search.run('Freedie Mercury and Queen')
print(output)

[snippet: Freddie Mercury was the charismatic front man of Queen who captivated audiences with his extraordinary vocal range and showmanship, leaving an enduring legacy in rock music., title: Freddie Mercury | Biography, Parents, Songs, & Facts, link: https://www.britannica.com/biography/Freddie-Mercury], [snippet: The best Freddie Mercury songs reveal an artist committed to exploring all avenues of expression: 'I can dream up all kinds of things.', title: Best Freddie Mercury Songs: 20 Essential Solo And Queen Tracks, link: https://www.udiscovermusic.com/stories/best-freddie-mercury-songs/], [snippet: Watch the first episode of Queen The Greatest Live, a series that celebrates the legendary performances of Freddie Mercury, the iconic lead singer of Queen., title: Queen The Greatest Live: Freddie Mercury - Part 1 (Episode 34), link: https://www.youtube.com/watch?v=wSw8-XlCcwI], [snippet: The Freddie Mercury Tribute Concert took place on Easter Monday, 20th April 1992 at Wembley Stadium

In [34]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
wrapper = DuckDuckGoSearchAPIWrapper(region='pt-pt', max_results=3, safesearch='moderate')
search = search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Brazil')

In [35]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\]'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches: 
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n' )
    print('-' * 50)
    

Snippet: Brazil, officially the Federative Republic of Brazil, is the largest and easternmost country in South America and in Latin America. Brazil is the world's fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo.
Title: Brazil - Wikipedia
Link: https://en.wikipedia.org/wiki/Brazil

--------------------------------------------------
Snippet: Mapa da densidade populacional do Brasil (2020) A população do Brasil, conforme censo realizado pelo Instituto Brasileiro de Geografia e Estatística (IBGE) em 2022, foi de 203 080 756 habitantes [ 187] (23,86 habitantes por quilômetro quadrado ), [ 188] com uma proporção de homens e mulheres de 0,94:1. [ 189]
Title: Brasil - Wikipédia, a enciclopédia livre
Link: https://pt.wikipedia.org/wiki/Brasil

--------------------------------------------------
Snippet: Brazil, country of South America that occupies half the continent's landmass. It is the fifth largest and fifth most-

### Wikipedia


In [37]:
!pip install -q wikipedia

In [38]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=10000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

'Page: NebulaGraph\nSummary: NebulaGraph is an open-source distributed graph database built for super large-scale graphs with milliseconds of latency. NebulaGraph adopts the Apache 2.0 license and  also  comes with a wide range of data visualization tools.'

In [39]:
output = wiki.invoke('Google Gemini')
print(output)

Page: Gemini (chatbot)
Summary: Gemini, formerly known as Bard, is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name and developed as a direct response to the meteoric rise of OpenAI's ChatGPT, it was launched in a limited capacity in March 2023 before expanding to other countries in May. It was previously based on PaLM, and initially the LaMDA family of large language models.
LaMDA had been developed and announced in 2021, but it was not released to the public out of an abundance of caution. OpenAI's launch of ChatGPT in November 2022 and its subsequent popularity caught Google executives off-guard and sent them into a panic, prompting a sweeping response in the ensuing months. After mobilizing its workforce, the company launched Bard in February 2023, which took center stage during the 2023 Google I/O keynote in May and was upgraded to the Gemini LLM in December. Bard and Duet AI were unified under the Gemini br